# Export best.pt model as .tflite model

In [1]:
from ultralytics import YOLO

lp_model = YOLO("runs/detect/train/weights/best.pt")

lp_model.export(format="tflite")

Ultralytics 8.3.146 🚀 Python-3.12.4 torch-2.7.0 CPU (Apple M3 Pro)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...


100%|██████████| 1.11M/1.11M [00:03<00:00, 312kB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /Users/nikoparas/vscode-workspace/SAIC/yolov8/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 70.69file/s]


ONNX: starting export with onnx 1.17.0 opset 19...


ONNX: slimming with onnxslim 0.1.54...
ONNX: export success ✅ 0.8s, saved as 'runs/detect/train/weights/best.onnx' (11.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.27.10...
Saved artifact at 'runs/detect/train/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 5, 8400), dtype=tf.float32, name=None)
Captures:
  13951804112: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  13951803920: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  13951805072: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  13951808336: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  13951806224: TensorSpec(shape=(3, 3, 16, 32), dtype=tf.float32, name=None)
  13951807952: TensorSpec(shape=(32,), dtype=tf.float32, name=None)
  13951804304: TensorSpec(shape=(1, 1, 32, 32), dtype=tf.float32, 

I0000 00:00:1749169432.397282 8603594 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1749169432.397396 8603594 single_machine.cc:374] Starting new session
W0000 00:00:1749169432.918073 8603594 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1749169432.918084 8603594 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1749169433.215421 8603594 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1749169433.215511 8603594 single_machine.cc:374] Starting new session


TensorFlow SavedModel: export success ✅ 12.8s, saved as 'runs/detect/train/weights/best_saved_model' (29.7 MB)

TensorFlow Lite: starting export with tensorflow 2.19.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'runs/detect/train/weights/best_saved_model/best_float32.tflite' (11.7 MB)

Export complete (12.9s)
Results saved to /Users/nikoparas/vscode-workspace/SAIC/yolov8/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640 data=data.yaml  
Visualize:       https://netron.app


W0000 00:00:1749169433.674671 8603594 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1749169433.674683 8603594 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


'runs/detect/train/weights/best_saved_model/best_float32.tflite'

# Initialize Tensor Flow Lite Interpreter

In [18]:
import tensorflow as tf
import os
import json
import cv2
import numpy as np

MODEL_PATH = "runs/detect/train/weights/best_saved_model/best_float32.tflite"
SAMPLE_DIR = "./sample"
RESULTS_DIR = "./results"
CONF_THRESH = 0.5

os.makedirs(RESULTS_DIR, exist_ok=True)

interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

print("Inputs:\n", input_details)
print("Outputs:\n", output_details)

Inputs:
 {'name': 'images', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
Outputs:
 {'name': 'Identity', 'index': 636, 'shape': array([   1,    5, 8400], dtype=int32), 'shape_signature': array([   1,    5, 8400], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


# Helper function to draw bounding boxes around detected license plates

In [19]:
def annotate_image(img_bgr: np.ndarray):
    """Run TFLite inference on img_bgr, draw boxes, return annotated BGR."""
    h0, w0, _ = img_bgr.shape
    h, w = input_details["shape"][1], input_details["shape"][2]

    # Preprocess: BGR→RGB, resize, float32, normalize
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(img_rgb, (w, h))
    tensor = np.expand_dims(resized, 0).astype(np.float32) / 255.0

    # Inference
    interpreter.set_tensor(input_details["index"], tensor)
    interpreter.invoke()

    out = interpreter.get_tensor(output_details["index"])[0]

    detections = out.T

    # Outputs: boxes [N,4], scores [N], classes [N]
    boxes = detections[:, :4]
    scores = detections[:, 4]

    confidence_scores = []

    # Draw boxes

    for (cx, cy, bw, bh), score in zip(boxes, scores):
        if score < CONF_THRESH:
            continue
        confidence_scores.append(float(score))
        # 1) get corner coords in normalized [0–1]
        x1 = cx - bw / 2
        y1 = cy - bh / 2
        x2 = cx + bw / 2
        y2 = cy + bh / 2

        # 2) scale to original image size
        pt1 = (int(x1 * w0), int(y1 * h0))
        pt2 = (int(x2 * w0), int(y2 * h0))

        # 3) draw
        cv2.rectangle(img_bgr, pt1, pt2, (0, 255, 0), 2)
        cv2.putText(
            img_bgr,
            f"{score:.2f}",
            (pt1[0], pt1[1] - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            1,
        )

    return img_bgr, confidence_scores

# Iterate through test data and draw bounding boxes

In [20]:
TEST_DIR = "./test/images"
TEST_RESULTS_DIR = "./test-results"

os.makedirs(TEST_RESULTS_DIR, exist_ok=True)

results = {}

for fname in os.listdir(TEST_DIR):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    in_path = os.path.join(TEST_DIR, fname)
    out_path = os.path.join(TEST_RESULTS_DIR, fname)

    img = cv2.imread(in_path)
    if img is None:
        print(f"[WARN] Could not load {in_path}, skipping.")
        continue

    annotated, confidences = annotate_image(img)
    cv2.imwrite(out_path, annotated)
    print(f"[OK] Wrote {out_path}")

    results[fname] = {
        "all_confidences": confidences,
        "max_confidence": max(confidences) if confidences else 0.0,
    }

with open("test_results_confidences.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ Wrote test_results_confidences.json")

[OK] Wrote ./test-results/Japanese-20License-20Plates-20-20Custom-20Japanese-20License-20P-20-1-_jpg.rf.511b035c29367e861cc81a4df187bc41.jpg
[OK] Wrote ./test-results/Cars430_png_jpg.rf.ed89a6a4fb4468882713f4b134a69e8e.jpg
[OK] Wrote ./test-results/cropngoaigiao5_jpg.rf.44569ee1a90ba7933113cacbc263e9cd.jpg
[OK] Wrote ./test-results/xemay1961_jpg.rf.4e8a3611dc795658e45cddfea3eb9864.jpg
[OK] Wrote ./test-results/CarLongPlateGen2239_jpg.rf.37522c937629f366ad7fb7d7c64b014c.jpg
[OK] Wrote ./test-results/32-cdmx2016dem_jpg.rf.131415f991a18caca53717c362178d93.jpg
[OK] Wrote ./test-results/xemayBigPlate197_jpg.rf.63a18be8d9eaf18409fdfa3d321eb5d3.jpg
[OK] Wrote ./test-results/002901d9d194c4fb_jpg.rf.ba35ca49985646e7c1e01fb118f038d8.jpg
[OK] Wrote ./test-results/0010f4c10f7ab07e_jpg.rf.1844f6dde3b97ed1c762db933bbacaf3.jpg
[OK] Wrote ./test-results/CarLongPlateGen1612_jpg.rf.051ada61adb5484aa58c13ef933c3cd7.jpg
[OK] Wrote ./test-results/brightnessquandoi4_jpg.rf.75e7f8c4fe30477ac626caa24f63ce32.j